In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

# Load Data

In [2]:
df_stock_sales_path = r'/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/dataset_experiment_2_stock_with_sales.csv'
df_warehouse_path = r'/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/dataset_experiment_2_warehouse.csv'

In [3]:
df_stock_sales = pd.read_csv(df_stock_sales_path).drop(columns=["Unnamed: 0"])
df_warehouse = pd.read_csv(df_warehouse_path).drop(columns=["Unnamed: 0"])

In [4]:
df_stock_sales['store'].unique()


array([ 100,   42, 4906,  109,   10,  201,   82,   46,  135,  213])

In [5]:
df_warehouse = df_warehouse.rename(columns={"stock": "warehouse_stock"})

# Load Dicts distribution of stores and years

In [6]:
dict_per_store_per_year_path = r"/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/dict_per_store_per_year.json"
with open(dict_per_store_per_year_path) as json_file:
    dict_per_store_per_year = json.load(json_file)

In [7]:
#dict_per_store_per_year chnage all keys to int
dict_per_store_per_year = {int(year): {int(store): dict_per_store_per_year[year][store] for store in dict_per_store_per_year[year]} for year in dict_per_store_per_year}

In [8]:
df_stock_sales.head()

,sku,store,date,average_price,average_dicounted_price,sales,item,sku_store,stock,year
0,100090812000001,100,2017-02-12,NaN,NaN,0.0,100090812000,"100090812000001,100",0.0,NaN
1,100090812000001,100,2017-02-13,NaN,NaN,0.0,100090812000,"100090812000001,100",0.0,NaN
2,100090812000001,100,2017-02-14,NaN,NaN,0.0,100090812000,"100090812000001,100",0.0,NaN
3,100090812000001,100,2017-02-15,NaN,NaN,0.0,100090812000,"100090812000001,100",0.0,NaN
4,100090812000001,100,2017-02-16,NaN,NaN,0.0,100090812000,"100090812000001,100",0.0,NaN


In [9]:
df_warehouse.head()

,sku,date,warehouse_stock
0,100090812000001,2017-09-12,0.0
1,100090812000001,2017-09-13,0.0
2,100090812000001,2017-09-14,0.0
3,100090812000001,2017-09-15,0.0
4,100090812000001,2017-09-16,0.0


In [10]:
sku_temp = 100630105000002

df_stock_sales_temp = df_stock_sales[df_stock_sales["sku"] == sku_temp]
df_warehouse_temp = df_warehouse[df_warehouse["sku"] == sku_temp]

In [11]:
df_stock_sales_temp.head()

,sku,store,date,average_price,average_dicounted_price,sales,item,sku_store,stock,year
7940,100630105000002,100,2020-09-02,NaN,NaN,0.0,100630105000,"100630105000002,100",0.0,NaN
7941,100630105000002,100,2020-09-03,NaN,NaN,0.0,100630105000,"100630105000002,100",0.0,NaN
7942,100630105000002,100,2020-09-04,NaN,NaN,0.0,100630105000,"100630105000002,100",0.0,NaN
7943,100630105000002,100,2020-09-05,NaN,NaN,0.0,100630105000,"100630105000002,100",0.0,NaN
7944,100630105000002,100,2020-09-06,NaN,NaN,0.0,100630105000,"100630105000002,100",0.0,NaN


# Expirement 3 stores: 100, 42, 4906

In [12]:
first_dist_store_100 =  dict_per_store_per_year[2020][100] / (dict_per_store_per_year[2020][100] + dict_per_store_per_year[2020][42] + dict_per_store_per_year[2020][4906])
first_dist_store_42 =  dict_per_store_per_year[2020][42] / (dict_per_store_per_year[2020][100] + dict_per_store_per_year[2020][42] + dict_per_store_per_year[2020][4906])
first_dist_store_4906 =  dict_per_store_per_year[2020][4906] / (dict_per_store_per_year[2020][100] + dict_per_store_per_year[2020][42] + dict_per_store_per_year[2020][4906])

In [13]:
print(first_dist_store_100, first_dist_store_42, first_dist_store_4906)

0.56062424969988 0.31892002083852405 0.12045572946159595


# Assumption:

##### 1. The time delivery ahead is 2 weeks (later we will change dynamically between stores for example 1 day - 14 days)
##### 2. the prior sales distribution of stores taking from 1 year before the experiment (for example 2019 if the experiment is in 2020)
##### 3. All the 126 stores going to be active in the optimization warehouse 
##### 4. Not all the time series of the stores are the same length, so there are some that starting earlier and some later (it depends on the store time shipping from the warehouse and also the time of the decision to start shipping to the store)
##### 5. Discount over time:
        
        5.1 after 20 weeks the discount is 0.8-0.85 % from the original price (the discount is the worst case for store because the store sales the product in the loss)
        
##### 6. Removing worst stores over time:
        
        6.1 6 weeks after store get the first delivery, if the store is in the bottom 10% of the stores from total stores, we remove it from the optimization warehouse
        6.2 8 weeks after store get the first delivery, if the store is in the bottom 20% of the stores from total stores, we remove it from the optimization warehouse
        6.3 12 weeks after store get the first delivery, if the store is in the bottom 30% of the stores from total stores, we remove it from the optimization warehouse
        6.4 14 weeks after store get the first delivery, if the store is in the bottom 40% of the stores from total stores, we remove it from the optimization warehouse

##### 7. There is no reordering to the warehouse what we have is what we have
##### 8. If the stock in the warehouse is 0 we can move the stock of sku from other store to the store that need it but it is not optimal
##### 10. The first allocation of the stock in store is given by the palmers (as 50% of the stock sku in the store they believe that the store can sell)



In [14]:
# dict_dynamic_current_decision = {}
# dict_dynamic_last_decision = {}
# dict_brain_current_global_sku_store = {}
# dict_brain_last_global_sku_store = {}
# dict_score_local_strategy = {}
# dict_score_global_strategy = {}

dict_dynamic_init = {}



In [15]:
df_stock_sales_temp["date"] = pd.to_datetime(df_stock_sales_temp["date"])
df_warehouse_temp["date"] = pd.to_datetime(df_warehouse_temp["date"])
df_stock_sales_temp = df_stock_sales_temp.sort_values(by=["date", "store"])
df_warehouse_temp = df_warehouse_temp.sort_values(by=["date"])

/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_52752/973854235.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock_sales_temp["date"] = pd.to_datetime(df_stock_sales_temp["date"])
/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_52752/973854235.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_warehouse_temp["date"] = pd.to_datetime(df_warehouse_temp["date"])


In [38]:
df_stock_sales_temp[df_stock_sales_temp['sku'] == sku_temp]['sales'].sum()

278.0

In [16]:
# current_year = "2020"
# last_year = "2019"

In [17]:
def create_dict_prior_initialize(years_list,stores_list):
    """
    This function create dict_prior_initialize only for the stores and years that we want to initialize and than we normalize the distribution for each year for the stores that we want to initialize
    :param years_list: 
    :param stores_list: 
    :return: 
    """
    dict_prior_initialize_only_stores_from_list = {
        year: {
            store: dict_per_store_per_year[year][store] / sum(dict_per_store_per_year[year].values())
            for store in dict_per_store_per_year[year] if store in stores_list
        }
        for year in dict_per_store_per_year if year in years_list
    }
    for year in dict_prior_initialize_only_stores_from_list:
        sum_year = sum([dict_prior_initialize_only_stores_from_list[year][store] for store in dict_prior_initialize_only_stores_from_list[year]])
        for store in dict_prior_initialize_only_stores_from_list[year]:
            dict_prior_initialize_only_stores_from_list[year][store] = dict_prior_initialize_only_stores_from_list[year][store] / sum_year
    return dict_prior_initialize_only_stores_from_list
    

In [18]:
dict_prior_initialize = create_dict_prior_initialize([2018,2019,2020,2021,2022], df_stock_sales['store'].unique())

In [19]:
dict_prior_initialize

{2018: {10: 0.05086257350686993,
  42: 0.13024534804864227,
  46: 0.08478953578031084,
  82: 0.15206748232583633,
  100: 0.27723121800767353,
  109: 0.14542515537471087,
  135: 0.03970532222253189,
  201: 0.02746114470982786,
  213: 0.029997305909348498,
  4906: 0.06221491411424803},
 2019: {10: 0.0530115348807852,
  42: 0.12402758272450413,
  46: 0.07187603553563822,
  82: 0.14563773215722783,
  100: 0.2896738358606978,
  109: 0.1445410506051473,
  135: 0.03818660943935115,
  201: 0.03244283843277105,
  213: 0.04060088681299607,
  4906: 0.0600018935508813},
 2020: {10: 0.06338357924939973,
  42: 0.14027676765664074,
  46: 0.08003148257001384,
  82: 0.15225209966823747,
  100: 0.24659021848505078,
  109: 0.1535970828808544,
  135: 0.04688511850796529,
  201: 0.0283841272055234,
  213: 0.03561714803781894,
  4906: 0.05298237573849541},
 2021: {10: 0.058781856485020226,
  42: 0.15787020637968643,
  46: 0.07740636477190105,
  82: 0.160181957158038,
  100: 0.24915639739741602,
  109: 0.151

In [30]:
dict_sku_static = {}
dict_sku_dynamic = {}
dict_sku_store_static = {}
dict_sku_store_dynamic = {}

In [31]:
for sku in df_stock_sales["sku"].unique():
    
    dict_sku_static[sku] = {}
    dict_sku_static[sku]["flag_stock_arrive_warehouse"] = False
    dict_sku_static[sku]["flag_remove_worst_stores_10_percent"] = False
    dict_sku_static[sku]["flag_remove_worst_stores_20_percent"] = False
    dict_sku_static[sku]["flag_remove_worst_stores_30_percent"] = False
    dict_sku_static[sku]["flag_remove_worst_stores_40_percent"] = False
    dict_sku_static[sku]["original_active_stores"] = []
    dict_sku_static[sku]["flag_6_weeks_after_first_delivery"] = False
    dict_sku_static[sku]["flag_8_weeks_after_first_delivery"] = False
    dict_sku_static[sku]["flag_12_weeks_after_first_delivery"] = False
    dict_sku_static[sku]["flag_14_weeks_after_first_delivery"] = False
    dict_sku_static[sku]["flag_20_weeks_after_first_delivery"] = False
    dict_sku_static[sku]["init_stock_in_warehouse"] = 0
    dict_sku_static[sku]["theoretical_remained_stock_per_store"] = {}
    
    
    dict_sku_dynamic[sku] = {}
    dict_sku_dynamic[sku]["current_remind_stock_in_warehouse"] = 0
    dict_sku_dynamic[sku]["current_remind_stock_in_warehouse_theoretical"] = 0 
    dict_sku_dynamic[sku]["store_rank"] = pd.Series()
    dict_sku_dynamic[sku]["date"] = None
    dict_sku_dynamic[sku]["prior_dynamic_per_stores"] = {}
    dict_sku_dynamic[sku]["strategy_global"] = {}
    dict_sku_dynamic[sku]["active_stores"] = []
    dict_sku_dynamic[sku]["10_percent_convert_map"] = 0

    
    df_stock_sales_sku = df_stock_sales[df_stock_sales["sku"] == sku]
    dict_sku_store_static[sku] = {}
    dict_sku_store_dynamic[sku] = {}    
    for store in df_stock_sales_sku["store"].unique():
        dict_sku_store_static[sku][store] = {}
        dict_sku_store_static[sku][store]["flag_stock_arrive_store"] = False
        dict_sku_store_static[sku][store]["prior_init_distribution"] = 0
        dict_sku_store_static[sku][store]["init_stock"] = 0
        dict_sku_store_static[sku][store]["time_delivery_ahead"] = 14
        dict_sku_store_static[sku][store]["flag_stock_not_enough_in_warehouse_for_store"] = False
        dict_sku_store_static[sku][store]["theoretical_remained_stock"] = 0
        
        dict_sku_store_dynamic[sku][store] = {}
        dict_sku_store_dynamic[sku][store]["date"] = None
        dict_sku_store_dynamic[sku][store]["cum_sum_of_sales"] = 0
        dict_sku_store_dynamic[sku][store]["prior_dynamic"] = {}
        dict_sku_store_dynamic[sku][store]["current_learning_rate"] = 0
        dict_sku_store_dynamic[sku][store]["next_learning_rate"] = 0
        dict_sku_store_dynamic[sku][store]["current_decision"] = "before_init"
        dict_sku_store_dynamic[sku][store]["next_decision"] = "before_init"   
        dict_sku_store_dynamic[sku][store]["current_stock"] = 0
        dict_sku_store_dynamic[sku][store]["current_remind_stock_in_store_theoretical"] = 0
        dict_sku_store_dynamic[sku][store]["current_sales"] = 0
        dict_sku_store_dynamic[sku][store]["history_data_sales"] = pd.DataFrame()
        dict_sku_store_dynamic[sku][store]["rolling_mean_of_7_days_sales"] = 0
        dict_sku_store_dynamic[sku][store]["rolling_mean_of_14_days_sales"] = 0
        dict_sku_store_dynamic[sku][store]["current_sales_rate_from_start"] = 0
        dict_sku_store_dynamic[sku][store]["number_of_days_from_first_delivery"] = 0
        dict_sku_store_dynamic[sku][store]["strategy_local"] = {}
        dict_sku_store_dynamic[sku][store]["factor_strategy_current"] = 0
        dict_sku_store_dynamic[sku][store]["palmers_stock"] = 0  
        dict_sku_store_dynamic[sku][store]["flag_active_store"] = False

/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_52752/3965792879.py:22: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dict_sku_dynamic[sku]["store_rank"] = pd.Series()
/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_52752/3965792879.py:22: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dict_sku_dynamic[sku]["store_rank"] = pd.Series()
/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_52752/3965792879.py:22: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dict_sku_dynamic[sku]["store_rank"] = pd.Series()
/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_52752/3965792879.py:22: FutureWarning: The default dt

In [32]:
dict_dynamic = {}

In [33]:
df_warehouse_temp.head()

,sku,date,warehouse_stock
6488,100630105000002,2020-09-01,3165.0
6489,100630105000002,2020-09-02,3165.0
6490,100630105000002,2020-09-03,3165.0
6491,100630105000002,2020-09-04,3165.0
6492,100630105000002,2020-09-05,3165.0


In [34]:
with open('/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/sku_sales_experiment2.json') as json_file:
    sku_sales_experiment = json.load(json_file)
#

In [35]:
# sku_sales_experiment2 keys to in
sku_sales_experiment = {int(sku): sku_sales_experiment[sku] for sku in sku_sales_experiment}

In [36]:
for date in df_warehouse_temp["date"].unique()[:80]:
    df_warehouse_temp_date = df_warehouse_temp[df_warehouse_temp["date"] <= date]
    df_stock_sales_temp_date = df_stock_sales_temp[df_stock_sales_temp["date"] <= date]
    
    for sku in [100630105000002]:

        df_warehouse_temp_date_sku = df_warehouse_temp_date[df_warehouse_temp_date["sku"] == sku]
        df_stock_sales_temp_date_sku = df_stock_sales_temp_date[df_stock_sales_temp_date["sku"] == sku]
        dict_sku_dynamic[sku]["date"] = date
        last_year = pd.to_datetime(date).year - 1
        if df_warehouse_temp_date_sku["warehouse_stock"].iloc[-1] > 0 and dict_sku_static[sku]["flag_stock_arrive_warehouse"]==False:
            dict_sku_static[sku]["flag_stock_arrive_warehouse"] = True
            # dict_sku_static[sku]["init_stock_in_warehouse"] = df_warehouse_temp_date_sku["warehouse_stock"].iloc[-1]
            dict_sku_static[sku]["init_stock_in_warehouse"] = sku_sales_experiment[sku]
            # dict_sku_dynamic[sku]["current_remind_stock_in_warehouse"] = df_warehouse_temp_date_sku["warehouse_stock"].iloc[-1]
            dict_sku_dynamic[sku]["current_remind_stock_in_warehouse"] = sku_sales_experiment[sku]
            dict_sku_dynamic[sku]["prior_dynamic_per_stores"] = create_dict_prior_initialize([last_year], [ 100,   42, 4906,  109,   10,  201,   82,   46,  135,  213])[last_year]

        
        if dict_sku_static[sku]["flag_stock_arrive_warehouse"]:
            
            if len(df_stock_sales_temp_date_sku) > 0:
                
                for store in df_stock_sales_temp_date_sku["store"].unique():
                    dict_sku_store_dynamic[sku][store]["date"] = date
                    df_stock_sales_temp_date_sku_store = df_stock_sales_temp_date_sku[df_stock_sales_temp_date_sku["store"] == store]
                    dict_sku_store_static[sku][store]["prior_init_distribution"] = create_dict_prior_initialize([2018,2019,2020,2021,2022], df_stock_sales['store'].unique())[last_year][store]
                    
                    
                    
                    dict_sku_store_dynamic[sku][store]["current_remind_stock_in_store_theoretical"] =  dict_sku_store_static[sku][store]["prior_init_distribution"] * dict_sku_static[sku]["init_stock_in_warehouse"]
                    
                    if df_stock_sales_temp_date_sku_store["stock"].iloc[-1] > 0 and dict_sku_store_static[sku][store]["flag_stock_arrive_store"] == False:
                        dict_sku_store_static[sku][store]["flag_stock_arrive_store"] = True
                        dict_sku_store_dynamic[sku][store]["prior_dynamic"] = dict_sku_store_static[sku][store]["prior_init_distribution"]
                        dict_sku_store_static[sku][store]["init_stock"] = df_stock_sales_temp_date_sku_store["stock"].iloc[-1]
                        dict_sku_store_dynamic[sku][store]["current_stock"] = dict_sku_store_static[sku][store]["init_stock"]
                        dict_sku_store_dynamic[sku][store]["number_of_days_from_first_delivery"] = 1
                        dict_sku_dynamic[sku]["current_remind_stock_in_warehouse"] -= dict_sku_store_static[sku][store]["init_stock"]
                        dict_sku_store_dynamic[sku][store]["flag_active_store"] = True
                        dict_sku_store_static[sku][store]["theoretical_remained_stock"] = dict_sku_store_static[sku][store]["prior_init_distribution"] * dict_sku_static[sku]["init_stock_in_warehouse"]
                        dict_sku_static[sku]["theoretical_remained_stock_per_store"][store] = dict_sku_store_static[sku][store]["theoretical_remained_stock"]
                        # dict_sku_dynamic[sku]["current_remind_stock_in_warehouse"] -= df_stock_sales_temp_date_sku_store["stock"].iloc[-1]
                        # print(dict_sku_dynamic[sku]["current_remind_stock_in_warehouse"])
                        
                    if dict_sku_store_static[sku][store]["flag_stock_arrive_store"]:
                        dict_sku_store_dynamic[sku][store]["number_of_days_from_first_delivery"] += 1
                        dict_sku_store_dynamic[sku][store]["history_data_sales"] = df_stock_sales_temp_date_sku_store.set_index("date")[["sales"]]
                        dict_sku_store_dynamic[sku][store]["cum_sum_of_sales"] = df_stock_sales_temp_date_sku_store["sales"].sum()   
                        dict_sku_store_dynamic[sku][store]["current_sales"] = df_stock_sales_temp_date_sku_store["sales"].iloc[-1]
                        dict_sku_store_dynamic[sku][store]["current_sales_rate_from_start"] = dict_sku_store_dynamic[sku][store]["cum_sum_of_sales"] / dict_sku_store_dynamic[sku][store]["number_of_days_from_first_delivery"]
                        dict_sku_store_dynamic[sku][store]["palmers_stock"] = df_stock_sales_temp_date_sku_store["stock"].iloc[-1]
                        dict_sku_store_dynamic[sku][store]["current_stock"] -= dict_sku_store_dynamic[sku][store]["current_sales"]

                    if dict_sku_store_dynamic[sku][store]["number_of_days_from_first_delivery"] >= 7:
                        dict_sku_store_dynamic[sku][store]["rolling_mean_of_7_days_sales"] = dict_sku_store_dynamic[sku][store]["history_data_sales"]["sales"].rolling(7).mean().iloc[-1]
                    if dict_sku_store_dynamic[sku][store]["number_of_days_from_first_delivery"] >= 14:
                        dict_sku_store_dynamic[sku][store]["rolling_mean_of_14_days_sales"] = dict_sku_store_dynamic[sku][store]["history_data_sales"]["sales"].rolling(14).mean().iloc[-1]
                    
                    if dict_sku_store_dynamic[sku][store]["number_of_days_from_first_delivery"] >= 6*7 and dict_sku_static[sku]["flag_6_weeks_after_first_delivery"] == False:
                        dict_sku_static[sku]["flag_6_weeks_after_first_delivery"] = True
                    if dict_sku_store_dynamic[sku][store]["number_of_days_from_first_delivery"] >= 8*7 and dict_sku_static[sku]["flag_8_weeks_after_first_delivery"] == False:
                        dict_sku_static[sku]["flag_8_weeks_after_first_delivery"] = True        
                    if dict_sku_store_dynamic[sku][store]["number_of_days_from_first_delivery"] >= 12*7 and dict_sku_static[sku]["flag_12_weeks_after_first_delivery"] == False:
                        dict_sku_static[sku]["flag_12_weeks_after_first_delivery"] = True
                    if dict_sku_store_dynamic[sku][store]["number_of_days_from_first_delivery"] >= 14*7 and dict_sku_static[sku]["flag_14_weeks_after_first_delivery"] == False:
                        dict_sku_static[sku]["flag_14_weeks_after_first_delivery"] = True   
                    if dict_sku_store_dynamic[sku][store]["number_of_days_from_first_delivery"] >= 20*7 and dict_sku_static[sku]["flag_20_weeks_after_first_delivery"] == False:
                        dict_sku_static[sku]["flag_20_weeks_after_first_delivery"] = True
                        

                        
                        
                list_active_stores = [store for store in dict_sku_store_static[sku] if dict_sku_store_static[sku][store]["flag_stock_arrive_store"]]
                dict_sku_static[sku]["original_active_stores"] = list_active_stores
                
                
                
                df_cumsum_all_stores = pd.DataFrame([[np.nan]*len(list_active_stores)],columns=list_active_stores)
                for active_store in list_active_stores:
                    df_cumsum_all_stores[active_store] = [dict_sku_store_dynamic[sku][active_store]["cum_sum_of_sales"]]
                dict_sku_dynamic[sku]["store_rank"] = df_cumsum_all_stores.rank(ascending=False, axis=1).T.sort_values(by=0, ascending=True)
                
                if dict_sku_static[sku]["flag_6_weeks_after_first_delivery"]== False:
                    dict_sku_dynamic[sku]["active_stores"] = dict_sku_static[sku]["original_active_stores"]
        
            
                if dict_sku_static[sku]["flag_6_weeks_after_first_delivery"]==True and dict_sku_static[sku]["flag_remove_worst_stores_10_percent"]==False:
                    dict_sku_dynamic[sku]["10_percent_convert_map"] = int(len(dict_sku_dynamic[sku]["active_stores"])*0.1)
                    dict_sku_dynamic[sku]["active_stores"] = [dict_sku_dynamic[sku]["store_rank"].index[i] for i in range(int(len(dict_sku_dynamic[sku]["active_stores"][:-dict_sku_dynamic[sku]['10_percent_convert_map']])))]
                    keep_prior_dynamic_list = []
                    for store in dict_sku_store_static[sku]:
                        if store not in dict_sku_dynamic[sku]["active_stores"]:
                            dict_sku_store_dynamic[sku][store]["flag_active_store"] = False
                            dict_sku_store_dynamic[sku][store]["prior_dynamic"] =  0
                            dict_sku_dynamic[sku]["prior_dynamic_per_stores"][store] = 0
                        else:
                            keep_prior_dynamic_list.append(dict_sku_store_dynamic[sku][store]["prior_dynamic"])
                    for store in dict_sku_dynamic[sku]["active_stores"]:
                        dict_sku_store_dynamic[sku][store]["prior_dynamic"] = dict_sku_store_dynamic[sku][store]["prior_dynamic"] / np.sum(keep_prior_dynamic_list)
                    dict_sku_static[sku]["flag_remove_worst_stores_10_percent"] = True
                    
                if dict_sku_static[sku]["flag_8_weeks_after_first_delivery"]==True and dict_sku_static[sku]["flag_remove_worst_stores_20_percent"]==False:   
                    dict_sku_dynamic[sku]["active_stores"] = [dict_sku_dynamic[sku]["store_rank"].index[i] for i in range(int(len(dict_sku_dynamic[sku]["active_stores"][:-dict_sku_dynamic[sku]["10_percent_convert_map"]])))]
                    keep_prior_dynamic_list = []
                    for store in dict_sku_store_static[sku]:
                        if store not in dict_sku_dynamic[sku]["active_stores"]:
                            dict_sku_store_dynamic[sku][store]["flag_active_store"] = False
                            dict_sku_store_dynamic[sku][store]["prior_dynamic"] =  0
                            dict_sku_dynamic[sku]["prior_dynamic_per_stores"][store] = 0
                        else:
                            keep_prior_dynamic_list.append(dict_sku_store_dynamic[sku][store]["prior_dynamic"])

                    for store in dict_sku_dynamic[sku]["active_stores"]:
                        dict_sku_store_dynamic[sku][store]["prior_dynamic"] = dict_sku_store_dynamic[sku][store]["prior_dynamic"] / np.sum(keep_prior_dynamic_list)
                    dict_sku_static[sku]["flag_remove_worst_stores_20_percent"] = True


                
                if dict_sku_static[sku]["flag_12_weeks_after_first_delivery"]==True and dict_sku_static[sku]["flag_remove_worst_stores_30_percent"]==False:
                    dict_sku_dynamic[sku]["active_stores"] = [dict_sku_dynamic[sku]["store_rank"].index[i] for i in range(int(len(dict_sku_dynamic[sku]["active_stores"][:-dict_sku_dynamic[sku]["10_percent_convert_map"]])))]
                    keep_prior_dynamic_list = []
                    for store in dict_sku_store_static[sku]:
                        if store not in dict_sku_dynamic[sku]["active_stores"]:
                            dict_sku_store_dynamic[sku][store]["flag_active_store"] = False
                            dict_sku_store_dynamic[sku][store]["prior_dynamic"] =  0
                            dict_sku_dynamic[sku]["prior_dynamic_per_stores"][store] = 0
                        else:
                            keep_prior_dynamic_list.append(dict_sku_store_dynamic[sku][store]["prior_dynamic"])
                    for store in dict_sku_dynamic[sku]["active_stores"]:
                        dict_sku_store_dynamic[sku][store]["prior_dynamic"] = dict_sku_store_dynamic[sku][store]["prior_dynamic"] / np.sum(keep_prior_dynamic_list)
                    dict_sku_static[sku]["flag_remove_worst_stores_30_percent"] = True
                
                if dict_sku_static[sku]["flag_14_weeks_after_first_delivery"]==True and dict_sku_static[sku]["flag_remove_worst_stores_40_percent"]==False:
                    dict_sku_dynamic[sku]["active_stores"] = [dict_sku_dynamic[sku]["store_rank"].index[i] for i in range(int(len(dict_sku_dynamic[sku]["active_stores"][:-dict_sku_dynamic[sku]["10_percent_convert_map"]])))]
                    keep_prior_dynamic_list = []
                    for store in dict_sku_store_static[sku]:
                        if store not in dict_sku_dynamic[sku]["active_stores"]:
                            dict_sku_store_dynamic[sku][store]["flag_active_store"] = False
                            dict_sku_store_dynamic[sku][store]["prior_dynamic"] =  0

                            dict_sku_dynamic[sku]["prior_dynamic_per_stores"][store] = 0
                        else:
                            keep_prior_dynamic_list.append(dict_sku_store_dynamic[sku][store]["prior_dynamic"])


                    for store in dict_sku_dynamic[sku]["active_stores"]:
                        dict_sku_store_dynamic[sku][store]["prior_dynamic"] = dict_sku_store_dynamic[sku][store]["prior_dynamic"] / np.sum(keep_prior_dynamic_list)
                    dict_sku_static[sku]["flag_remove_worst_stores_40_percent"] = True

                
                
                for store in dict_sku_dynamic[sku]["active_stores"]:
                        dict_sku_dynamic[sku]["prior_dynamic_per_stores"][store] = dict_sku_store_dynamic[sku][store]["prior_dynamic"]


                df_active_stores_dist = pd.DataFrame([[np.nan]*len(dict_sku_dynamic[sku]["active_stores"])],columns=dict_sku_dynamic[sku]["active_stores"])
                df_active_stores_sales_rate_from_start = pd.DataFrame([[np.nan]*len(dict_sku_dynamic[sku]["active_stores"])],columns=dict_sku_dynamic[sku]["active_stores"])
                for active_store in dict_sku_dynamic[sku]["active_stores"]:
                    df_active_stores_dist[active_store] = dict_sku_store_dynamic[sku][active_store]["prior_dynamic"]
                    df_active_stores_sales_rate_from_start[active_store] = dict_sku_store_dynamic[sku][active_store]["current_sales_rate_from_start"]

                a1 = df_active_stores_sales_rate_from_start.sub(df_active_stores_sales_rate_from_start.min(axis=1), axis=0)
                a2 = a1.divide(a1.max(axis=1), axis=0).fillna(0)
                a2_normalized = a2 * df_active_stores_dist.sum(axis=1).iloc[0]
                new_dist_list = []
                for active_store in dict_sku_dynamic[sku]["active_stores"]:
                    old_dist = dict_sku_store_dynamic[sku][active_store]["prior_dynamic"]
                    new_dist = old_dist + a2_normalized[active_store].iloc[0] * 0.05
                    new_dist_list.append(new_dist)

                new_dist_list_normalized = np.multiply([new_dist / sum(new_dist_list) for new_dist in  new_dist_list] ,df_active_stores_dist.sum(axis=1).iloc[0])
                new_dist_list_normalized_dict = {dict_sku_dynamic[sku]["active_stores"][i]: new_dist_list_normalized[i] for i in range(len(dict_sku_dynamic[sku]["active_stores"]))}

                for active_store in dict_sku_dynamic[sku]["active_stores"]:
                     dict_sku_store_dynamic[sku][active_store]["prior_dynamic"] = new_dist_list_normalized_dict[active_store]
                     dict_sku_dynamic[sku]['prior_dynamic_per_stores'][active_store] = new_dist_list_normalized_dict[active_store]



238.0
198.0
148.0
108.0
96.0
84.0
66.0
54.0
42.0
4.0


In [39]:
df_stock_sales_temp_date_sku_store["stock"].iloc[-1]

7.0

In [28]:
dict_sku_static[sku]

{'flag_stock_arrive_warehouse': True,
 'flag_remove_worst_stores_10_percent': True,
 'flag_remove_worst_stores_20_percent': True,
 'flag_remove_worst_stores_30_percent': False,
 'flag_remove_worst_stores_40_percent': False,
 'original_active_stores': [100, 42, 4906, 109, 10, 201, 82, 46, 135, 213],
 'flag_6_weeks_after_first_delivery': True,
 'flag_8_weeks_after_first_delivery': True,
 'flag_12_weeks_after_first_delivery': False,
 'flag_14_weeks_after_first_delivery': False,
 'flag_20_weeks_after_first_delivery': False,
 'init_stock_in_warehouse': 278.0,
 'theoretical_remained_stock_per_store': {82: 40.48728953970934,
  100: 80.52932636927399,
  109: 40.182412068230946,
  213: 11.287046534012907,
  10: 14.737206696858285,
  46: 19.981537878907424,
  135: 10.615877424139619,
  201: 9.019109084310351,
  4906: 16.680526407145003}}

In [ ]:
new_dist_list_normalized

In [ ]:
dict_sku_store_[sku]

In [ ]:
dict_sku_store_static[sku]

In [ ]:
dict_sku_dynamic[sku]["active_stores"]

In [ ]:
dict_sku_dynamic[sku]['prior_dynamic_per_stores'][201]

In [ ]:
sum(np.sum(dict_sku_dynamic[sku]['prior_dynamic_per_stores'].values()))

In [ ]:
list_active_stores

In [ ]:
new_dist_list_normalized

In [ ]:
7*8

In [ ]:
dict_sku_dynamic[sku]

In [ ]:
dict_prior_initialize[last_year]

In [ ]:
print(df_warehouse_temp_date_sku)

In [ ]:
dict_sku_store_dynamic[sku]

In [ ]:
keep_prior_dynamic_list

In [ ]:
dict_sku_dynamic[sku]["prior_dynamic_per_stores"]

In [ ]:
dict_sku_store_dynamic[sku]

In [ ]:
dict_sku_dynamic

In [ ]:
dict_sku_static[sku]["original_active_stores"]

In [ ]:
# change it to str values in list in dict_sku_static[sku]["original_active_stores"]



In [ ]:
dict_sku_static[sku]["original_active_stores"]

In [ ]:
create_dict_prior_initialize([str(last_year)], dict_sku_static[sku]["original_active_stores"])

In [ ]:
dict_sku_dynamic[sku]["prior_dynamic_per_stores"][last_year]

In [ ]:
dict_sku_dynamic[sku]["prior_dynamic_per_stores"][last_year]

In [ ]:
# dict_sku_store_dynamic[sku][store]["date"] = None
#         dict_sku_store_dynamic[sku][store]["cum_sum_of_sales"] = 0
#         dict_sku_store_dynamic[sku][store]["prior_dynamic"] = {}
#         dict_sku_store_dynamic[sku][store]["current_learning_rate"] = 0
#         dict_sku_store_dynamic[sku][store]["next_learning_rate"] = 0
#         dict_sku_store_dynamic[sku][store]["current_decision"] = "before_init"
#         dict_sku_store_dynamic[sku][store]["next_decision"] = "before_init"   
#         dict_sku_store_dynamic[sku][store]["current_stock"] = 0
#         dict_sku_store_dynamic[sku][store]["current_remind_stock_in_store_theoretical"] = 0
#         dict_sku_store_dynamic[sku][store]["current_sales"] = 0
#         dict_sku_store_dynamic[sku][store]["history_data_sales"] = pd.DataFrame()
#         dict_sku_store_dynamic[sku][store]["rolling_sum_of_7_days_sales"] = 0
#         dict_sku_store_dynamic[sku][store]["rolling_sum_of_14_days_sales"] = 0
#         dict_sku_store_dynamic[sku][store]["rolling_sum_of_21_days_sales"] = 0
#         dict_sku_store_dynamic[sku][store]["rolling_sum_of_28_days_sales"] = 0
#         dict_sku_store_dynamic[sku][store]["current_sales_rate"] = 0
#         dict_sku_store_dynamic[sku][store]["current_sales_rate_7_days"] = 0
#         dict_sku_store_dynamic[sku][store]["current_sales_rate_14_days"] = 0
#         dict_sku_store_dynamic[sku][store]["number_of_days_from_first_delivery"] = 0
#         dict_sku_store_dynamic[sku][store]["strategy_local"] = {}
#         dict_sku_store_dynamic[sku][store]["factor_strategy_current"] = 0
#         dict_sku_store_dynamic[sku][store]["palmers_decision"] = 0  

In [ ]:
dict_sku_store_dynamic

In [ ]:
dict_sku_store_dynamic[sku][active_store]["prior_dynamic"]

In [ ]:
dict_sku_store_dynamic[sku][active_store]["prior_dynamic"]

In [ ]:
df_active_stores_sales_rate_from_start

In [ ]:
np.sum(new_dist_list)

In [ ]:
new_dist_list_normalized.sum()

In [ ]:
dict_sku_dynamic

In [ ]:
dict_sku_store_static[sku]

In [ ]:
dict_sku_store_dynamic[sku]

In [ ]:
df_cumsum_all_stores

In [ ]:
# rank from 1 to len(list_active_stores)
df_cumsum_all_stores.rank(ascending=False, axis=1)

In [ ]:
dict_sku_static[sku]

In [ ]:
dict_sku_dynamic[sku]["prior_dynamic_per_stores"]

In [ ]:
dict_sku_dynamic

In [ ]:
dict_sku_dynamic[sku]["store_rank"]

In [ ]:
[store for store in dict_sku_store_static[sku] if dict_sku_dynamic[sku]["active_stores"] and dict_sku_dynamic[sku]["store_rank"][store].iloc[0] < 1 * len(dict_sku_dynamic[sku]["active_stores"])]